In [6]:
#your_project_directory/
#│
#├── main.py (или название вашего основного файла FastAPI)
#│
#├── templates/ (директория для шаблонов)
#│   └── index.html (ваш HTML-шаблон)
#│
#└── static/ (директория для статических файлов, если они есть)
#    ├── css/
#    ├── js/
#    └── images/

In [7]:
!pip install fastapi uvicorn

In [8]:
pip install fastapi uvicorn python-multipart

In [9]:
from google.colab import drive

# Подключение к Google Диску
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import shutil
import requests
from urllib.parse import urlencode
import os


In [11]:
# #загружаем проект

# Путь к исходному файлу
source_folder = '/content/drive/MyDrive/Colab Notebooks/UII/internship_safe _head/fast_api/static.zip'

# Путь к целевой папке на Google Диске
destination_folder = '/content/'

# Копирование файла на Google Диск
shutil.copy(source_folder, destination_folder)



#разархивировать полученные данные
import zipfile

# Путь к вашему архиву .zip
zip_path = '/content/static.zip'

# Папка, в которую вы хотите извлечь файлы
extracted_folder = '/content/'

# Создайте папку для извлечения файлов (если она не существует)
os.makedirs(extracted_folder, exist_ok=True)

# Распаковка архива
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

print("Архив успешно разархивирован.")

Архив успешно разархивирован.


In [12]:
#загружаем проект


# Путь к исходному файлу
source_folder = '/content/drive/MyDrive/Colab Notebooks/UII/internship_safe _head/fast_api/templates.zip'

# Путь к целевой папке на Google Диске
destination_folder = '/content/'

# Копирование файла на Google Диск
shutil.copy(source_folder, destination_folder)


#разархивировать полученные данные
import zipfile

# Путь к вашему архиву .zip
zip_path = '/content/templates.zip'

# Папка, в которую вы хотите извлечь файлы
extracted_folder = '/content/'

# Создайте папку для извлечения файлов (если она не существует)
os.makedirs(extracted_folder, exist_ok=True)

# Распаковка архива
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

print("Архив успешно разархивирован.")

Архив успешно разархивирован.


In [13]:
#загружаем веса yolov8

# Путь к исходному файлу
source_folder = '/content/drive/MyDrive/Colab Notebooks/UII/internship_safe _head/fast_api/weight.zip'

# Путь к целевой папке на Google Диске
destination_folder = '/content/'

# Копирование файла на Google Диск
shutil.copy(source_folder, destination_folder)

# Распаковка архива
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)



In [14]:
script = '''

from fastapi import FastAPI, Request, File, UploadFile  # Импорт необходимых классов и функций из FastAPI
from fastapi.responses import HTMLResponse, StreamingResponse  # Импорт необходимых классов откликов (responses) из FastAPI
from fastapi.staticfiles import StaticFiles  # Импорт класса для обслуживания статических файлов из FastAPI
from fastapi.templating import Jinja2Templates  # Импорт класса шаблонов Jinja2 из FastAPI
from fastapi.middleware.cors import CORSMiddleware  # Импорт класса промежуточного программного обеспечения (middleware) CORS из FastAPI
from fastapi.responses import Response
import asyncio  # Import asyncio module

import os
os.system('pip install ultralytics')
os.system('pip install lapx')
from ultralytics import YOLO
from collections import defaultdict

import cv2
import matplotlib.pyplot as plt
import datetime

app = FastAPI()

# Directory for uploaded files
UPLOAD_DIRECTORY = "/content/files"
if not os.path.exists(UPLOAD_DIRECTORY):
    print("Creating upload directory...")
    os.makedirs(UPLOAD_DIRECTORY)

# Глобальная переменная для хранения пути к последнему загруженному файлу
last_uploaded_file = None

app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# Разрешаем запросы CORS от любого источника
origins = ["*"]  # Для простоты можно разрешить доступ со всех источников
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


my_map = {}
def defineHurdHut(image, box, save_dir, id_person):
    int_list = [int(x) for x in box]
    (x_min, y_min, x_max, y_max) = int_list
    #print(x_min, y_min, x_max, y_max)
    # Вырезаем кусок изображения по bounding box
    cropped_image = image[y_min:y_max, x_min:x_max]

    # Подаем вырезанный кусок на обработку (например, детекция объектов, классификация и т.д.)
    # В данном случае просто выводим размеры вырезанного куска
    #print("Размеры вырезанного куска:", cropped_image.shape)
    # Сохраняем вырезанное изображение в файл
    if cropped_image is not None and cropped_image.size != 0:
      cv2.imwrite('/content/cropped_image.jpg', cropped_image)
    else:
        print("Cropped image is empty.")
        return
    # Отображение изображения и вырезанного куска (для визуализации)
    # Рисуем bounding box
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Отображаем оригинальное изображение
    plt.clf()
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis("off")
    plt.show()

    # Берем новую модель
    model = YOLO('weight/best.pt')
    # Обнаруживаем только класс без каски в режиме генератора (stream)
    result = model.predict(cropped_image, stream=True, classes=1, device='cpu')
    for i, r in enumerate(result):
      # При детекции головы без каски
      if (len(r.boxes) > 0):
        if id_person not in my_map:
            my_map[id_person] = 1
            current_time = datetime.datetime.utcnow()
            formatted_time = current_time.strftime("%d%m%Y_%H%M%S")  # Форматирует время как 'ДДММГГГГ_ЧЧММСС'
            file_name = save_dir+'/person_'+formatted_time+'.jpg'
            cv2.imwrite(file_name, cropped_image)
            # Получаем изображение, вырезаем bb
            #boxes = r.boxes.xyxy
            #numbers_list = boxes.tolist()
            #for box_d in numbers_list:
              #int_list_d = [int(x) for x in box_d]
              #(x_min_d, y_min_d, x_max_d, y_max_d) = int_list_d
              # Рисуем bounding box
              #cv2.rectangle(cropped_image, (x_min_d, y_min_d), (x_max_d, y_max_d), (0, 255, 0), 2)

    #plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
    #plt.title("Cropped Image")
    #plt.axis("off")
    #plt.show()
    # Вывод результата
    #print("Стандартный вывод:")
    #print(result.stdout)

    # Вывод ошибок
    #print("Стандартная ошибка:")
    #print(result.stderr)

#функция  распознавания человека без каски на видео
#video_file - текущий видеофайл
#save_dir - Куда будем сохранять изображения
#file_yolo - веса для yolo8
def mainProgram(video_file_path, save_dir, file_yolo):
    # Если нет каталога, создадим его
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # Берем стандартную модель
    model = YOLO(file_yolo)

    # Open the video file
    cap = cv2.VideoCapture(video_file_path)

    # Получение размеров кадра и частоты кадров из исходного видео
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Создание объекта VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_video.avi', fourcc, fps, (frame_width, frame_height))

    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True)

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Plot the tracks
            n = 0
            for box, track_id in zip(boxes, track_ids):
                n=n+1
                x, y, w, h = box
                x1 = int(x-w/2)
                x2 = int(x + w/2)
                y1 = int(y - h/2)
                y2 = int(y + h/2)

                # Отображение результатов трекинга
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                # вырезаем каждый bb и далее отправляем к следующей нейросети
                box_yolo = [x1, y1, x2, y2]
                defineHurdHut(frame, box_yolo, save_dir, track_id)

           #если в кадре не оказалось людей, то можно обнулять список id трекера
            if (n==0):
                my_map.clear()

              # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
        # Запись обработанного кадра в файл
        out.write(frame)

    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()

last_uploaded_file = "/content/files/file369_267.avi"


@app.get("/")
@app.get("/index.html", response_class=HTMLResponse)
async def home(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.get("/favicon.ico")
async def favicon():
    return Response(content="", status_code=204)

@app.post("/upload-file/")
async def upload_file(file: UploadFile = File(...)):
    print("begin save file")
    file_location = os.path.join(UPLOAD_DIRECTORY, file.filename)
    with open(file_location, "wb+") as file_object:
        file_object.write(file.file.read())
    last_uploaded_file = file_location
    print(f"File {file.filename} saved at {file_location}")
    #return {"info": f"File '{file.filename}' saved at '{file_location}'"}

@app.post("/start-detection/")
async def start_detection():
    try:
        if last_uploaded_file is None:
            raise BaseException()

        print(f"Starting detection on {last_uploaded_file}...")

        video_file_path = last_uploaded_file
        save_dir = "exam"
        file_yolo = "/content/weight/yolov8n.pt"

        # запускаем обработку видео
        mainProgram(video_file_path, save_dir, file_yolo)

        return {"message": "Обработка видео начата"}
    except Exception as e:
        print(f"Error during detection: {str(e)}")
        raise BaseException()


@app.get("/get-exam-files-list")
async def get_exam_files_list():
    exam_dir = "exam"
    files = [file for file in os.listdir(exam_dir) if os.path.isfile(os.path.join(exam_dir, file))]
    print(files)
    return {"files": files}

'''

with open('main.py', 'w') as f:
  f.write(script)

In [15]:
script = '''
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" href="static/css/index.css" >
    <title>File Viewer App</title>
</head>
<body>
    <h1>Контроль сотрудников без каски</h1>
    <form action="/upload-file/" method="post" enctype="multipart/form-data" id="upload-form">
        <!-- Кастомная кнопка для загрузки файла -->
        <div class="file-upload-wrapper">
            <label class="custom-file-upload">
                <input type="file" name="file" class="file-upload-input">
                Загрузить файл
            </label>
        </div>
        <br>
        <br>
        <div id="menu" class="file-upload-wrapper">
            <!-- Additional menu options here -->
            <button class="custom-file-upload" onclick="beginDetection()">
                Начать детекцию
            </button>
        </div>
    </form>

    <div class="iframe-container">
        <iframe id="file-viewer" style="width:80%; height:80vh;"></iframe>
        <iframe id="file-viewer_error" style="width:20%; height:80vh;">
            <div id="file-list">
              <!-- Здесь будут отображаться созданные файлы -->
            </div>
        </iframe>
    </div>

    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="static/js/index.js"></script>
</body>
</html>
'''

with open('templates/index.html', 'w') as f:
  f.write(script)

In [16]:
script = '''
    function beginDetection() {
      fetch('/start-detection/', { method: 'POST' })
      .then(response => response.json())
      .then(data => {
          console.log(data.message);
          // Обновление интерфейса пользователя, если необходимо
      })
      .catch(error => {
          console.error('Ошибка при запуске детекции:', error);
      });
    }

    function updateFileList() {
        $.ajax({
            url: "/get-exam-files-list",  // URL-эндпоинт FastAPI для получения списка файлов
            type: "GET",
            dataType: "json",
            success: function(data) {
                var fileList = $("#file-list");
                fileList.empty();  // Очищаем список перед обновлением

                // Добавляем каждый файл в список
                data.files.forEach(function(fileName) {
                    var fileLink = $("<a>").attr("href", "/exam-files/" + fileName).text(fileName);
                    var listItem = $("<li>").append(fileLink);
                    fileList.append(listItem);
                });
            },
            error: function() {
                console.error("Failed to fetch file list");
            }
        });
    }

    // Вызываем функцию обновления списка файлов при загрузке страницы
    $(document).ready(function() {
        updateFileList();
    });

'''

with open('static/js/index.js', 'w') as f:
  f.write(script)

In [17]:
script = '''

        /* Стили для скрытия стандартной кнопки выбора файла */
        .custom-file-upload {
            border: 1px solid #ccc;
            display: inline-block;
            padding: 6px 12px;
            cursor: pointer;
            color: rgb(255, 254, 254);
            background-color: rgb(0, 0, 0);
        }
        .file-upload-wrapper {
            position: relative;
            overflow: hidden;
            display: inline-block;
        }
        .file-upload-input {
            position: absolute;
            left: 0;
            top: 0;
            opacity: 0;
            cursor: pointer;
        }

        button{
            color: rgb(255, 254, 254);
            background-color: rgb(0, 0, 0);
        }

        button:hover {
            box-shadow: 10px 4px 8px 10px rgba(131, 93, 93, 0.2); /* Тень при наведении */
        }

        input:hover{
            box-shadow: 10px 4px 8px 10px rgba(131, 93, 93, 0.2); /* Тень при наведении */
        }

        h1{
          color: rgb(0, 0, 0);
          text-decoration: solid;
          font-style: italic; /* Делает текст наклонным */;
        }

        body {
            background-image: url('../../static/images/fon.jpg'); /* Путь к изображению */
            background-size: cover; /* Растягивает изображение, чтобы оно покрывало весь экран */
            background-repeat: no-repeat; /* Предотвращает повторение изображения */
            background-attachment: fixed; /* Фиксирует изображение на фоне */
        }

                /* Стили для контейнера фреймов */
        .iframe-container {
            display: flex; /* Активирует Flexbox для контейнера */
        }

        iframe {
            border: black solid 2px; /* Опционально, убирает рамку у iframe */
        }

'''

with open('static/css/index.css', 'w') as f:
  f.write(script)

In [18]:
!nohup uvicorn main:app --host 0.0.0.0 --port 8000  --reload &

nohup: appending output to 'nohup.out'


In [19]:
!pip install ngrok
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.1 MB/s eta 0:00:00


In [21]:
from pyngrok import ngrok
ngrok.set_auth_token('2bK23tcE9xZ9IiqyDLflWQOZFvj_7iKitcEQH7xqqoiq6cnGf')
ngrok.kill()
connection = ngrok.connect(8000)
print(f"Public URL: {connection.public_url}")

ERROR:pyngrok.process.ngrok:t=2024-01-27T09:30:19+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session obj=csess id=b5e92802d21a err="authentication failed: Your account is limited to 1 simultaneous ngrok agent session.\nYou can run multiple tunnels on a single agent session using a configuration file.\nTo learn more, see https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config/\n\nActive ngrok agent sessions in region 'us':\n  - ts_2bX2DBwXP7HdFjEClsOa7sOEe4R (34.16.155.212)\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2024-01-27T09:30:19+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent session.\nYou can run multiple tunnels on a single agent session using a configuration file.\nTo learn more, see https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config/\n\nActive ngrok agent sessions in region 'us':\n  - ts_2bX2DBwXP7HdFjEClsOa7sOEe4R (34.16.155.212)\

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent session.\nYou can run multiple tunnels on a single agent session using a configuration file.\nTo learn more, see https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config/\n\nActive ngrok agent sessions in region 'us':\n  - ts_2bX2DBwXP7HdFjEClsOa7sOEe4R (34.16.155.212)\r\n\r\nERR_NGROK_108\r\n.

In [ ]:
active_tunnels = ngrok.get_tunnels()
print(active_tunnels)

In [ ]:
stop

In [ ]:
!pkill uvicorn

In [ ]:
!kill -9 $(lsof -t -i:8000)